# Smart Journey Planner: Constraint-Based Routing
1. Compute optimal route through the list of user selected points
2. Constraint is that the start point is fixed and start and end point should be the same and route should traverse all points

### a. Data preparation:
1. Merge points of interest layers (Amenities, Shops, Tourism and Historic places) into a single file 
2. Create a new column with all the place names

In [ ]:
#Define input output directories
import os

# Define input and output directories
input_dir = os.path.join(os.getcwd(), "Data")
output_dir = os.path.join(os.getcwd(), "Output")

# Create both directories if they do not exist
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [ ]:
import geopandas as gpd
import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

# ------------------------------------------------------
# STEP 1 - Merge All POI GeoPackages
# ------------------------------------------------------

# Data (Points of interest features in Varanasi)
paths = [
    "https://iudx-cat-sandbox-dev.s3.ap-south-1.amazonaws.com/simplerouting-data/amenity_varanasi.gpkg",
    "https://iudx-cat-sandbox-dev.s3.ap-south-1.amazonaws.com/simplerouting-data/shops_varanasi.gpkg",
    "https://iudx-cat-sandbox-dev.s3.ap-south-1.amazonaws.com/simplerouting-data/shops_varanasi.gpkg",
    "https://iudx-cat-sandbox-dev.s3.ap-south-1.amazonaws.com/simplerouting-data/tourism_varanasi.gpkg",
]

gdfs = [gpd.read_file(p) for p in paths]
all_gdf = pd.concat(gdfs, ignore_index=True)

# capture the original CRS (your sample uses EPSG:7755)
orig_crs = all_gdf.crs

# unify name field and drop duplicates
all_gdf["place_name"] = all_gdf["name_en"].combine_first(all_gdf["name"])
gdf_final = all_gdf.drop_duplicates(subset=["place_name"], keep="first").reset_index(drop=True)

#View the gdf
gdf_final.head()

,amenity,name_en,name,geometry,shop,tourism,place_name
0,cinema,None,BLW Cinema Hall,POINT (4292552.656 4142789.017),NaN,NaN,BLW Cinema Hall
1,hospital,None,Mata Anandmayee Hosiptal,POINT (4296844.402 4143602.286),NaN,NaN,Mata Anandmayee Hosiptal
2,place_of_worship,Sankat Mochan Hanuman Temple (Mandir),संकट मोचन हनुमान मंदिर,POINT (4296317.006 4142405.269),NaN,NaN,Sankat Mochan Hanuman Temple (Mandir)
3,place_of_worship,None,Baba Keenaram Ashram,POINT (4296617.145 4143922.253),NaN,NaN,Baba Keenaram Ashram
4,place_of_worship,None,St Marys Church,POINT (4294171.52 4148243.884),NaN,NaN,St Marys Church


### b. Enable widgets for the users to select start point and stops to create route

- Displays widgets to select start point and stops. Limitation is set for the user to select 2 - 15 points, including the start point.
- STOPS - Select the points from the scrollview widget using ctrl (for random selection) or shift (for continuous selection)
- START POINT - have 3 options
    1. If "None" is selected, first point in the sorted STOPS list will act as start point
    2. If "Current" is selected, the current location based on ip_address will be fetched
    3. If "Pick on Map" is selected, an interactive map is displayed, click on the desired location and click on 'Use This Point' button
- Finally click on 'Confirm slection', the selected stops, including the start point is saved as "input_file.geojson" in 'Data' folder.

In [ ]:
import geopandas as gpd
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from shapely.geometry import Point
import requests
import ipyleaflet as ipyl

import warnings
warnings.filterwarnings("ignore")

def get_current_location_ipinfo(return_gdf=True):
    try:
        data = requests.get("https://ipinfo.io/json", timeout=5).json()
        city, region, country = data.get("city", "?"), data.get("region", "?"), data.get("country", "?")
        lat_s, lon_s = data.get("loc", ",").split(",")
        lat, lon = float(lat_s), float(lon_s)
        place_name = f"{city}, {region}, {country}".strip(", ")
        print(f"📍 Current location: {place_name} | lat={lat:.6f}, lon={lon:.6f}")
        if return_gdf:
            return gpd.GeoDataFrame([{"place_name": place_name,
                                      "geometry": Point(lon, lat)}],
                                    crs="EPSG:4326")
        return lat, lon
    except Exception as e:
        print("⚠️ ipinfo failed:", e)
        return None

_picked_gdf = None


def manual_pick_widget(center_latlon):
    global _picked_gdf
    _picked_gdf = None
    m = ipyl.Map(center=center_latlon, zoom=12,
                 basemap=ipyl.basemaps.OpenStreetMap.Mapnik)
    marker = ipyl.Marker(location=center_latlon, draggable=True)
    m.add_layer(marker)

    info = widgets.HTML("Drag the red marker or click on the map to move it.")
    use_btn = widgets.Button(description="Use This Point", button_style="info")
    status  = widgets.Output()

    def handle_click(**kwargs):
        if kwargs.get("type") == "click":
            marker.location = kwargs["coordinates"]
    m.on_interaction(handle_click)

    def on_use(_):
        with status:
            clear_output()
            lat, lon = marker.location
            print(f"Picked: lat={lat:.6f}, lon={lon:.6f}")
            gdf = gpd.GeoDataFrame([{"place_name": "Selected on map",
                                     "geometry": Point(lon, lat)}],
                                   crs="EPSG:4326")
            global _picked_gdf
            _picked_gdf = gdf
    use_btn.on_click(on_use)

    out_map = widgets.Output()
    with out_map:
        display(m)
    return widgets.VBox([info, out_map, use_btn, status])

#Get the  centre of the points of interest layer to display map zoomed to that extent
centroid = gdf_final.to_crs("EPSG:4326").geometry.unary_union.centroid
poi_center = (centroid.y, centroid.x)

# Widgets
place_names = sorted(gdf_final["place_name"].dropna().unique())
selector = widgets.SelectMultiple(options=place_names, description="Select Places:", rows=10)
location_mode = widgets.ToggleButtons(options=[("None", "none"), ("Current", "current"), ("Pick on Map", "manual")], description="Start Point:")
confirm = widgets.Button(description="Confirm Selection", button_style="success")
map_box = widgets.VBox()
out = widgets.Output()


# Show map when manual selected
def on_mode(change):
    map_box.children = []
    if change["new"] == "manual":
        map_box.children = [manual_pick_widget(poi_center)]

location_mode.observe(on_mode, names="value")

def on_confirm(_):
    with out:
        clear_output()
        chosen = list(selector.value)
        if not 2 <= len(chosen) <= 15:
            print("Choose 2–15 places.")
            return
        sel_gdf = gdf_final[gdf_final["place_name"].isin(chosen)].copy().reset_index(drop=True)

        if location_mode.value == "current":
            curr = get_current_location_ipinfo(True)
            if curr is not None:
                sel_gdf = pd.concat([curr.to_crs(orig_crs), sel_gdf], ignore_index=True)
        elif location_mode.value == "manual":
            if _picked_gdf is None:
                print("Pick a point then click 'Use This Point'.")
                return
            sel_gdf = pd.concat([_picked_gdf.to_crs(orig_crs), sel_gdf],
                                ignore_index=True)

        geom_only = sel_gdf[["geometry"]]
        input_file = os.path.join(input_dir, "input_file.geojson")
        geom_only.to_file(input_file, driver="GeoJSON")
        print("Wrote ▶ input_file.geojson")

        sel_gdf[["place_name"]].to_csv("./Data/input_file_names.csv", index=False)
        print("Wrote ▶ input_file_names.csv")

        display(geom_only)

confirm.on_click(on_confirm)

display(widgets.VBox([selector, location_mode, confirm, out, map_box]))


### c. Fetch road data using SDK

- Input client id and secret (which is made available for download at the time of registration)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
import os
import time

# ----------------------------------------
# Create interactive widgets for user input
# ----------------------------------------

# Widget for client ID (text input)
client_id_widget = widgets.Text(
    description="Client ID:",
    placeholder="Enter Client ID",
    layout=widgets.Layout(width='90%')
)

# Widget for client secret (hidden input)
client_secret_widget = widgets.Password(
    description="Client Secret:",
    placeholder="Enter Client Secret",
    layout=widgets.Layout(width='90%')
)

# Button to execute the command
execute_button = widgets.Button(
    description="Fetch Resource",
    button_style="success"   # Makes the button green
)

# Output area to display logs and messages
output = widgets.Output()

# ------------------------------------------------
# Debounce logic to prevent multiple rapid clicks
# ------------------------------------------------

last_click_time = 0

def on_click(b):
    """
    Handler function triggered when the button is clicked.
    It reads the widget values and runs the GDI SDK command.
    """
    global last_click_time

    # Debounce: ignore clicks occurring within 0.5 seconds
    current_time = time.time()
    if current_time - last_click_time < 0.5:
        return
    last_click_time = current_time

    with output:
        clear_output()

        # Read user inputs from widgets
        client_id = client_id_widget.value.strip()
        client_secret = client_secret_widget.value.strip()

        # Save the road to 'Data' folder created
        Road_data = os.path.join(input_dir, "Road.geojson")

        # Prepare the GDI SDK command to fetch vector data
        cmd = [
            "gdi", "get-vector-data",
            "--client-id", client_id,
            "--client-secret", client_secret,
            "--role", "consumer",
            "--resource-id", "a4395596-14e6-4e17-83c4-989bc23fb3d2",
            "--store-artifact", "local",
            "--config-path", "config.json",
            "--file-path", Road_data
        ]

        # Print the command for debugging purposes
        print("Executing command:")
        print(" ".join(cmd))
        print()

        try:
            # Run the command and capture output
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)

            # Print success message and output file path
            print("[DONE] Collection download complete.")
            print(f"GDI vector resource successfully downloaded to:\n{Road_data}")

        except subprocess.CalledProcessError as e:
            # Print error message if command fails
            print("[ERROR] Failed to fetch resource.")
            print(e.stderr)

# ------------------------------------------------
# Attach the click handler to the button
# ------------------------------------------------

execute_button.on_click(on_click)

# ------------------------------------------------
# Display all widgets and the output area
# ------------------------------------------------

display(
    client_id_widget,
    client_secret_widget,
    execute_button,
    output
)


Text(value='', description='Client ID:', layout=Layout(width='90%'), placeholder='Enter Client ID')

Password(description='Client Secret:', layout=Layout(width='90%'), placeholder='Enter Client Secret')

Button(button_style='success', description='Fetch Resource', style=ButtonStyle())

Output()

### d. Define functions to compute optimal route using travelling salesman
1. Load road network as graph
2. Load the points to the graph
3. Snap the points to the graph
4. Compute all pairwise shortest paths/distances using A* heuristic algorithm
5. Solve travelling salesman problem
6. Save the route and ordered points as geojson

In [5]:
import os
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from tqdm import tqdm
from scipy.spatial import KDTree
from networkx.algorithms.approximation import traveling_salesman_problem
import folium


In [6]:

def load_road_network(file_path):
    """Load a road network GeoPackage and build a weighted NetworkX graph."""
    gdf = gpd.read_file(file_path)
    print("Road network loaded! CRS:", gdf.crs)
    gdf = gdf.explode(ignore_index=True).dropna(subset=["geometry"])
    G = nx.Graph()
    for _, row in gdf.iterrows():
        if row.geometry.geom_type == "LineString":
            pts = list(row.geometry.coords)
            for u, v in zip(pts, pts[1:]):
                G.add_edge(u, v, weight=Point(u).distance(Point(v)))
    print(f"Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    return gdf, G

def load_sample_points(file_path, target_crs):
    """Read sample points and reproject to match target CRS if needed."""
    pts = gpd.read_file(file_path)
    print("Sample points CRS:", pts.crs)
    if pts.crs != target_crs:
        pts = pts.to_crs(target_crs)
        print("Reprojected sample points to", target_crs)
    return [list(geom.coords)[0] for geom in pts.geometry]

def snap_points_to_graph(graph_nodes, sample_pts):
    """Snap each sample point to its nearest graph node via KDTree."""
    tree = KDTree(graph_nodes)
    snapped = [graph_nodes[tree.query(pt)[1]] for pt in tqdm(sample_pts, desc="Snapping")]
    return snapped

def compute_shortest_paths(G, snapped_nodes):
    """Compute all pairwise shortest paths/distances using A* heuristic."""
    shortest = {}
    heuristic = lambda a, b: Point(a).distance(Point(b))
    for i in tqdm(range(len(snapped_nodes)), desc="Pairwise A*"):
        for j in range(i + 1, len(snapped_nodes)):
            u, v = snapped_nodes[i], snapped_nodes[j]
            try:
                path = nx.astar_path(G, u, v, heuristic=heuristic, weight="weight")
                dist = nx.astar_path_length(G, u, v, heuristic=heuristic, weight="weight")
            except nx.NetworkXNoPath:
                path, dist = None, float("inf")
            shortest[(i, j)] = {"path": path, "distance": dist}
    return shortest

def solve_tsp(shortest_paths):
    """Form a complete graph of stops and solve the TSP cycle."""
    T = nx.Graph()
    for (i, j), info in shortest_paths.items():
        T.add_edge(i, j, weight=info["distance"])
    return traveling_salesman_problem(T, cycle=True)

def extract_tsp_path(G, tsp_order, snapped_nodes):
    """Recover the full node-by-node route and ordered stop points."""
    full_route, ordered_pts = [], []
    for idx in range(len(tsp_order) - 1):
        start, end = snapped_nodes[tsp_order[idx]], snapped_nodes[tsp_order[idx+1]]
        ordered_pts.append(Point(start))
        try:
            segment = nx.shortest_path(G, start, end, weight="weight")
            full_route.extend(segment)
        except nx.NetworkXNoPath:
            print(f"No path between {start} and {end}")
    ordered_pts.append(Point(snapped_nodes[tsp_order[-1]]))
    return full_route, ordered_pts

def save_route(route_nodes, ordered_pts, crs, route_fp, pts_fp):
    """Write the TSP route and ordered points out as GeoJSON files."""
    gdf_route = gpd.GeoDataFrame(geometry=[LineString(route_nodes)], crs=crs)
    gdf_route.to_file(route_fp, driver="GeoJSON")
    gdf_pts = gpd.GeoDataFrame(geometry=ordered_pts, crs=crs)
    gdf_pts["order"] = range(1, len(ordered_pts) + 1)
    gdf_pts.to_file(pts_fp, driver="GeoJSON")
    print("Saved route →", route_fp)
    print("Saved points →", pts_fp)

def plot_route_map(route_fp, pts_fp, zoom=12):
    """Load saved GeoJSONs, reproject to EPSG:4326, and display Folium map."""
    route = gpd.read_file(route_fp).to_crs(epsg=4326)
    pts   = gpd.read_file(pts_fp).to_crs(epsg=4326)
    centre = route.geometry.iloc[0].centroid
    m = folium.Map(location=[centre.y, centre.x],
                   zoom_start=zoom, control_scale=True)
    coords = [(lat, lon) for lon, lat in route.geometry.iloc[0].coords]
    folium.PolyLine(coords, weight=4, opacity=0.8, tooltip="TSP route").add_to(m)
    n = len(pts)
    for _, row in pts.iterrows():
        colour = "green" if row.order in (1, n) else "red"
        folium.Marker(
            [row.geometry.y, row.geometry.x],
            tooltip=f"Stop {row.order}",
            icon=folium.Icon(color=colour, icon="info-sign")
        ).add_to(m)
    return m

In [ ]:
# --------------------------------------------
# MAIN PIPELINE: from input_file.geojson → TSP route + map
# --------------------------------------------
import os, shutil
import pandas as pd
import geopandas as gpd

# your helper functions defined earlier:
#   load_road_network, load_sample_points,
#   snap_points_to_graph, compute_shortest_paths,
#   solve_tsp, extract_tsp_path, save_route, plot_route_map

# ——— INPUT/ OUTPUT PARAMETERS ———
# Input Road data to construct network
road_fp    = "./Data/Road.geojson"
# Read the point files generated after selecting the points
samples_fp = "./Data/input_file.geojson" 
# Read csv with place names
names_fp = "./Data/input_file_names.csv" 
#-----------------------------------------------
# Define output paths - > route as linestring and selected points ordered according to the route
output_dir = "./Output"
os.makedirs(output_dir, exist_ok=True) 
route_fp   = "./Output/tsp_route.geojson"
pts_fp     = "./Output/tsp_ordered_points.geojson"
pts_fp_named = "./Output/tsp_ordered_points_with_names.geojson"
# 1) Load road network
roads_gdf, G = load_road_network(road_fp)

# 2) Read your selected points & reproject to match roads CRS
sample_pts = load_sample_points(samples_fp, roads_gdf.crs)
names      = pd.read_csv(names_fp)["place_name"].tolist()

# 3) Snap the selected points to nearest road nodes
graph_nodes   = list(G.nodes)
snapped_nodes = snap_points_to_graph(graph_nodes, sample_pts)

# 4) Compute all-pair shortest paths/distances
shortest = compute_shortest_paths(G, snapped_nodes)

# 5) Solve the Travelling Salesman Problem
tsp_order = solve_tsp(shortest)

# 6) Expand the TSP order into a full route + ordered stops
full_route_nodes, ordered_pts = extract_tsp_path(G, tsp_order, snapped_nodes)

# 7) Save the GeoJSON outputs
save_route(full_route_nodes, ordered_pts, roads_gdf.crs, route_fp, pts_fp)

# 8) save a second points layer WITH place-names
ordered_names = [names[i] for i in tsp_order]
# pts_fp_named  = "/content/tsp_ordered_points_with_names.geojson"
gdf_pts_named = gpd.GeoDataFrame(
    {"order": range(1, len(ordered_pts) + 1),
     "name" : ordered_names},
    geometry=ordered_pts,
    crs=roads_gdf.crs
)
gdf_pts_named.to_file(pts_fp_named, driver="GeoJSON")
print("Saved points+names →", pts_fp_named)


Road network loaded! CRS: EPSG:4326
Graph: 725125 nodes, 747623 edges
Sample points CRS: EPSG:7755
Reprojected sample points to EPSG:4326


Pairwise A*: 100%|██████████| 6/6 [00:16<00:00,  2.69s/it]


Saved route → ./Output/tsp_route.geojson
Saved points → ./Output/tsp_ordered_points.geojson
Saved points+names → ./Output/tsp_ordered_points_with_names.geojson


### e. Visualize the output on interactive map
If there are n stops
- green marker denotes start and end point (stop 1 and n)
- red marker denotes mid stops with stop numbers 2 to n-1.

In [16]:
def plot_route_map(route_fp, pts_fp, zoom=12):
    """Load saved GeoJSONs, reproject to EPSG:4326, and display Folium map."""
    route = gpd.read_file(route_fp).to_crs(epsg=4326)
    pts   = gpd.read_file(pts_fp_named).to_crs(epsg=4326)
    centre = route.geometry.iloc[0].centroid
    m = folium.Map(location=[centre.y, centre.x],
                   zoom_start=zoom, control_scale=True)
    coords = [(lat, lon) for lon, lat in route.geometry.iloc[0].coords]
    folium.PolyLine(coords, weight=4, opacity=0.8, tooltip="TSP route").add_to(m)
    n = len(pts)
    for _, row in pts.iterrows():
        colour = "green" if row.order in (1, n) else "red"
        folium.Marker(
            [row.geometry.y, row.geometry.x],
            tooltip=f"stop_{row.order}:{row["name"]}",
            icon=folium.Icon(color=colour, icon="info-sign")
        ).add_to(m)
    return m

In [17]:
# Render an interactive Folium map
m = plot_route_map(route_fp, pts_fp, zoom=10)
m  # display in notebook